## Molkit

In [1]:
# import libraries
import numpy as np
from moha import HamHeisenberg

J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]
                 ])

mu = np.zeros(2)

ham = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham.generate_one_body_integral(dense=True)
two_body = ham.generate_two_body_integral(dense=True)

In [2]:
import numpy as np
from moha import HamHeisenberg
from moha.molkit.hamiltonians import MolHam
from moha.molkit.utils.tools import from_geminal
from moha.molkit.utils.spinops import antisymmetrize_two_body

def from_geminal(two_body_gem, n_orb):
    """
    Inverse of MolHam.to_geminal().

    Parameters
    ----------
    two_body_gem : (n_gem, n_gem) ndarray
        Matrix in the geminal basis.
    n_orb : int
        Number of spin orbitals.

    Returns
    -------
    V : (n_orb, n_orb, n_orb, n_orb) ndarray
        Fully antisymmetrised two-electron tensor V_{ijkl}.
    """
    n_gem = n_orb * (n_orb - 1) // 2
    if two_body_gem.shape != (n_gem, n_gem):
        raise ValueError(f"Shape mismatch: got {two_body_gem.shape}, expected ({n_gem},{n_gem})")

    # Generate flattened pair list exactly like to_geminal
    pairs = [(i, j) for i in range(n_orb) for j in range(i + 1, n_orb)]
    V = np.zeros((n_orb, n_orb, n_orb, n_orb))

    for A, (i, j) in enumerate(pairs):
        for B, (k, l) in enumerate(pairs):
            val = 0.25 * two_body_gem[A, B]  # undo the factor 0.5 * 4 = 2

            # Apply antisymmetric filling
            V[i, j, k, l] = val
            V[j, i, k, l] = -val
            V[i, j, l, k] = -val
            V[j, i, l, k] = val
            V[k, l, i, j] = val
            V[l, k, i, j] = -val
            V[k, l, j, i] = -val
            V[l, k, j, i] = val

    return V


J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]])
mu = np.zeros(2)

ham_hei  = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham_hei.generate_one_body_integral(dense=True)
two_body = ham_hei.generate_two_body_integral(dense=True)
Molecular_Hamiltonian = MolHam(one_body=one_body, two_body=two_body)

h1_spin, h2_spin = Molecular_Hamiltonian.spinize_H()
h2_spin = antisymmetrize_two_body(h2_spin)

reduced = Molecular_Hamiltonian.to_reduced(n_elec=2)

# Energies Tests

In [3]:
import numpy as np
from pyscf import fci
from moha.molkit.hamiltonians import MolHam


h_sp = np.array([[1.0, 0.2],
                 [0.2, 1.5]])

V_sp = np.zeros((2, 2, 2, 2))
V_sp[0, 0, 0, 0] = 0.7
V_sp[1, 1, 1, 1] = 0.6

N_ELEC, n_spin  = 2, 2                  
toy = MolHam(h_sp, V_sp)

h_spin, V_spin = toy.spinize_H()           
norb = h_spin.shape[0]
e_fci, ci_vec = fci.direct_spin1.kernel(h_spin, V_spin, norb, N_ELEC, ecore=0.0)
rdm1s, rdm2s = fci.direct_spin1.make_rdm12s(ci_vec, norb, N_ELEC)

rdm1 = np.block([
    [rdm1s[0], np.zeros_like(rdm1s[0])],
    [np.zeros_like(rdm1s[1]), rdm1s[1]]
])

n = norb // 2  # spatial orbitals
rdm2 = np.zeros((2*n, 2*n, 2*n, 2*n))
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                # αααα
                rdm2[i, j, k, l] = rdm2s[0][i, k, j, l]
                # ββββ
                rdm2[i+n, j+n, k+n, l+n] = rdm2s[2][i, k, j, l]
                # αβαβ
                rdm2[i, j+n, k, l+n] = rdm2s[1][i, k, j, l]
                # βαβα (hermitian transpose of αβαβ)
                rdm2[i+n, j, k+n, l] = rdm2s[1][i, k, j, l]

H = toy.to_reduced(n_elec=N_ELEC)  # H = h↑ + 0.5 * V
E_spin = np.einsum('pqrs,pqrs', H, rdm2)

H_gem = toy.to_geminal(H, type='h2')
rdm_gem = toy.to_geminal(rdm2, type='rdm2')
E_gem   = np.einsum('pq,pq', H_gem, rdm_gem)

print("E_spin-orbital :", E_spin)
print("E_geminal      :", E_gem)
assert np.allclose(E_spin, E_gem, atol=1e-10), "Energies do not match!"

E_spin-orbital : 1.004059262354666e-12
E_geminal      : 1.0040474662350293e-12
